In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum
from scipy import stats

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
def gurobi_network_Q(C, sample_Q_sp, sample_Q_pc, D, S, R, M):
    # input:
    # C (unit cost for building a facility): p*1 numpy array
    # sample_Q_sp (unit flow cost): s*p*g*k numpy array
    # sample_Q_pc (unit flow cost): p*c*g*k numpy array
    # D (demand): c*g*k numpy array
    # S (supply): s*g numpy array
    # R (unit processing require): p*g numpy array
    # M (processing capacity): p*1 numpy array
    # output:
    # x (open facility): p*1 numpy array
    # y_sp (flow supplier --> facility): s*p*g numpy array
    # y_pc (flow facility --> consumer): p*c*g numpy array
    s, p, g, k = sample_Q_sp.shape
    c, g = D.shape
    model = Model("network")
    x = model.addVars(p, vtype=GRB.BINARY, name="x")
    y_sp = model.addVars(s, p, g, lb=0, vtype=GRB.CONTINUOUS, name="y_sp")
    y_pc = model.addVars(p, c, g, lb=0, vtype=GRB.CONTINUOUS, name="y_pc")

    model.setObjective(sum(C[j] * x[j] for j in range(p)) \
                       + 1/k * sum(sample_Q_sp[i, j, l, a] * y_sp[i, j, l] for i in range(s) for j in range(p) for l in range(g) for a in range(k))\
                        + 1/k * sum(sample_Q_pc[j, i, l, a] * y_pc[j, i, l] for j in range(p) for i in range(c) for l in range(g) for a in range(k)), GRB.MINIMIZE)
    
    for l in range(g):
        for j in range(p):
            model.addConstr(sum(y_sp[i, j, l] for i in range(s)) - sum(y_pc[j, i, l] for i in range(c)) == 0, f"flow_{j}_{l}")
        for i in range(c):
            model.addConstr(sum(y_pc[j, i, l] for j in range(p)) >= D[i, l], f"demand_{i}_{l}")
        for i in range(s):
            model.addConstr(sum(y_sp[i, j, l] for j in range(p)) <= S[i, l], f"supply_{i}_{l}")
    
    for j in range(p):
        model.addConstr(sum(R[j, l] * (sum(y_sp[i, j, l] for i in range(s))) for l in range(g)) <= M[j] * x[j], f"capacity_{j}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        x_opt = np.array([x[i].X for i in range(p)])
        y_sp_opt = np.array([[[y_sp[i, j, l].X for i in range(s)] for j in range(p)] for l in range(g)])
        y_pc_opt = np.array([[[y_pc[j, i, l].X for i in range(c)] for j in range(p)] for l in range(g)])
        return x_opt, y_sp_opt, y_sp_opt
    else:
        print("No optimal solution found.")
        return None
    

In [3]:
def sample_func(distribution_name, **dist_params):
     if hasattr(np.random, distribution_name):
        # Access the distribution function dynamically
        dist_func = getattr(np.random, distribution_name)
        # Generate samples
        samples = dist_func(**dist_params)
        return samples
     else:
        raise ValueError(f"Unsupported distribution: {distribution_name}")

In [4]:
def majority_vote(sample_S, sample_D, B, k, eval_func, *eval_args):
    x_count = {}
    for _ in range(B):
        # choose k samples from total n samples
        sample_sk = sample_S[:,:,np.random.choice(sample_S.shape[2], k, replace=False)]
        sample_Dk = sample_D[:,:,np.random.choice(sample_D.shape[2], k, replace=False)]
        x_k = tuple(eval_func(sample_sk, sample_Dk, *eval_args))
        x_count[x_k] = x_count.get(x_k, 0) + 1
    
    x_max = max(x_count, key=x_count.get)
    return x_max

In [ ]:
def gurobi_first_stage(sample_S, sample_D, C, Q_sp, Q_pc, R, M, H):
    # input:
    # C (unit cost for building a facility): p*1 numpy array
    # Q_sp (unit flow cost): s*p*g numpy array
    # Q_pc (unit flow cost): p*c*g numpy array
    # sample_S (supply): s*g*k numpy array
    # sample_D (demand): c*g*k numpy array
    # R (unit processing require): p*g numpy array
    # M (processing capacity): p*1 numpy array
    # H (multiplier): c*g numpy array
    # output:
    # x (open facility): p*1 numpy array
    # y_sp (flow supplier --> facility): s*p*g*k numpy array 
    # y_pc (flow facility --> consumer): p*c*g*k numpy array
    # z (multiplier): c*g*k numpy array
    s, p, g = Q_sp.shape
    c, g, k = sample_D.shape
    model = Model("network")
    x = model.addVars(p, vtype=GRB.BINARY, name="x")
    y_sp = model.addVars(s, p, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_sp")
    y_pc = model.addVars(p, c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_pc")
    z = model.addVars(c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="z")

    model.setObjective(sum(C[j] * x[j] for j in range(p)) \
                       + 1/k * sum(Q_sp[i, j, l] * y_sp[i, j, l, a] for i in range(s) for j in range(p) for l in range(g) for a in range(k))\
                        + 1/k * sum(Q_pc[j, i, l] * y_pc[j, i, l, a] for j in range(p) for i in range(c) for l in range(g) for a in range(k))\
                            + 1/k * sum(H[i, l] * z[i, l, a] for i in range(c) for l in range(g) for a in range(k)), GRB.MINIMIZE)
    for a in range(k):
        for l in range(g):
            for j in range(p):
                model.addConstr(sum(y_sp[i, j, l, a] for i in range(s)) - sum(y_pc[j, i, l, a] for i in range(c)) == 0, f"flow_{j}_{l}")
            for i in range(c):
                model.addConstr(sum(y_pc[j, i, l, a] + z[i, l, a] for j in range(p)) >= sample_D[i, l, a], f"demand_{i}_{l}")
            for i in range(s):
                model.addConstr(sum(y_sp[i, j, l, a] for j in range(p)) <= sample_S[i, l, a], f"supply_{i}_{l}")
        for j in range(p):
            model.addConstr(sum(R[j, l] * sum(y_sp[i, j, l, a] for i in range(s)) for l in range(g)) <= M[j] * x[j], f"capacity_{j}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        x_opt = np.array([x[i].X for i in range(p)])
        # y_sp_opt = np.array([[[[y_sp[i, j, l, a].X for a in range(k)] for l in range(g)] for j in range(p)] for i in range(s)])
        # y_pc_opt = np.array([[[[y_pc[j, i, l, a].X for a in range(k)] for l in range(g)] for j in range(p)] for i in range(c)])
        # z_opt = np.array([[[z[i, l, a].X for a in range(k)] for l in range(g)] for i in range(c)])
        return x_opt #, y_sp_opt, y_pc_opt, z_opt
    else:
        print("No optimal solution found.")
        return None
    

In [6]:
def gurobi_second_stage(sample_S, sample_D, C, Q_sp, Q_pc, R, M, H, x):
    s, p, g = Q_sp.shape
    c, g, k = sample_D.shape
    model = Model("second_stage")
    y_sp = model.addVars(s, p, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_sp")
    y_pc = model.addVars(p, c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="y_pc")
    z = model.addVars(c, g, k, lb=0, vtype=GRB.CONTINUOUS, name="z")

    model.setObjective(1/k * sum(Q_sp[i, j, l] * y_sp[i, j, l, a] for i in range(s) for j in range(p) for l in range(g) for a in range(k))\
                        + 1/k * sum(Q_pc[j, i, l] * y_pc[j, i, l, a] for j in range(p) for i in range(c) for l in range(g) for a in range(k))\
                            + 1/k * sum(H[i, l] * z[i, l, a] for i in range(c) for l in range(g) for a in range(k)), GRB.MINIMIZE)
    for a in range(k):
        for l in range(g):
            for j in range(p):
                model.addConstr(sum(y_sp[i, j, l, a] for i in range(s)) - sum(y_pc[j, i, l, a] for i in range(c)) == 0, f"flow_{j}_{l}")
            for i in range(c):
                model.addConstr(sum(y_pc[j, i, l, a] + z[i, l, a] for j in range(p)) >= sample_D[i, l, a], f"demand_{i}_{l}")
            for i in range(s):
                model.addConstr(sum(y_sp[i, j, l, a] for j in range(p)) <= sample_S[i, l, a], f"supply_{i}_{l}")
        for j in range(p):
            model.addConstr(sum(R[j, l] * sum(y_sp[i, j, l, a] for i in range(s)) for l in range(g)) <= M[j] * x[j], f"capacity_{j}")
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        return model.ObjVal + sum(C[j] * x[j] for j in range(p)) 
    else:
        print("No optimal solution found.")
        return None
    

In [21]:
# Run script - parameter setup

s = 3 # numbe of suppliers
p = 5 # number of facilities
c = 3 # number of consumers
g = 4 # number of proucts

# Randomly generate input parameters based on the given dimensions
C = np.random.rand(p)           # Unit cost for building a facility
Q_sp = np.random.rand(s, p, g)  # Unit flow cost from supplier to facility
Q_pc = np.random.rand(p, c, g)  # Unit flow cost from facility to customer
R = np.random.rand(p, g)        # Unit processing requirement
M = np.random.rand(p)           # Processing capacity
H = np.random.rand(c, g)       # Multiplier

sample_number = np.array([2**i for i in range(10, 11)])
number_of_iterations = 2

# Generate samples for evaluation of optimal SAA and majority vote
large_number_sample = 1
large_sample_S = sample_func('pareto', size=(s,g,large_number_sample), a=2.8)  
large_sample_D = sample_func('pareto', size=(c,g,large_number_sample), a=2.8) 

# x, y_sp, y_pc, z= gurobi_first_stage(C, Q_sp, Q_pc, sample_D, sample_S, R, M, H)

In [23]:
# SAA and majority vote comparison

SAA_list = []
majority_list = []
for ite in sample_number:
    SAA_intermediate = []
    majority_intermediate = []
    for _ in range(number_of_iterations):
        n = ite
        sample_S = sample_func('pareto', size=(s,g,n), a=2.8)  
        sample_D = sample_func('pareto', size=(c,g,n), a=2.8) 

        SAA = majority_vote(sample_S,sample_D, 1, n, gurobi_first_stage, C, Q_sp, Q_pc, R, M, H)
        SAA_intermediate.append(SAA)

        majority = majority_vote(sample_S, sample_D, 100, int(n/10), gurobi_first_stage, C, Q_sp, Q_pc, R, M, H)
        majority_intermediate.append(majority)
        
    SAA_list.append(SAA_intermediate)
    majority_list.append(majority_intermediate)

SAA_obj_list = []
majority_obj_list = []
for i in range(len(sample_number)):
    SAA_obj = 0
    majority_obj = 0
    for j in range(number_of_iterations):
        SAA_obj += gurobi_second_stage(large_sample_S, large_sample_D, C, Q_sp, Q_pc, R, M, H, SAA_list[i][j])
        SAA_obj = SAA_obj/number_of_iterations
        majority_obj += gurobi_second_stage(large_sample_S, large_sample_D, C, Q_sp, Q_pc, R, M, H, majority_list[i][j])
        majority_obj = majority_obj/number_of_iterations

    SAA_obj_list.append(SAA_obj)
    majority_obj_list.append(majority_obj)
        

fig2, ax2 = plt.subplots()
ax2.plot(sample_number, SAA_obj_list, marker = 'o', markeredgecolor = 'none', color = 'blue',linestyle = 'solid', linewidth = 2, label = 'SAA')
ax2.plot(sample_number, majority_obj_list, marker = 's', markeredgecolor = 'none', color = 'red',linestyle = 'solid', linewidth = 2, label = 'Majority Vote')
ax2.set_xlabel('Number of samples', size = 20)
ax2.set_ylabel('Objective', size = 20)
ax2.legend(loc = 'lower right')
plt.show()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 50176 rows, 135173 columns and 324608 nonzeros
Model fingerprint: 0x68a37e3f
Variable types: 135168 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [6e-02, 5e+00]
  Objective range  [6e-06, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-07, 7e+01]
Found heuristic solution: objective 1.030734e+10
Presolve removed 5990 rows and 36530 columns
Presolve time: 0.68s
Presolved: 44186 rows, 98643 columns, 242027 nonzeros
Found heuristic solution: objective 2.6641506
Variable types: 98638 continuous, 5 integer (5 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 5.654598e-01, 15798 iterations, 0.25 seconds (0.18 wo

KeyboardInterrupt: 

In [20]:
print(SAA_list)
print(majority_list)


[[(0.0, 0.0, 1.0, 1.0, 0.0), (0.0, 0.0, 1.0, 1.0, 0.0)]]
[[(0.0, 0.0, 1.0, 1.0, 0.0), (0.0, 0.0, 1.0, 1.0, 0.0)]]


In [137]:
large_number_sample = 10000
sample_D = sample_func('pareto', size=(s,g,large_number_sample), a=2.8)  # Unit flow cost from supplier to facility
sample_S = sample_func('pareto', size=(c,g,large_number_sample), a=2.8)  # Unit flow cost from facility to customer
expect_second_stage = gurobi_second_stage(x, C, Q_sp, Q_pc, sample_D, sample_S, R, M, H)

print(expect_second_stage)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 490000 rows, 1320000 columns and 3120000 nonzeros
Model fingerprint: 0x7844cdff
Coefficient statistics:
  Matrix range     [7e-02, 5e+00]
  Objective range  [4e-07, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-07, 2e+02]
Presolve removed 102676 rows and 480284 columns
Presolve time: 1.93s
Presolved: 387324 rows, 839716 columns, 1910921 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.07s

Barrier statistics:
 AA' NZ     : 1.183e+06
 Factor NZ  : 3.357e+06 (roughly 500 MB of memory)
 Factor Ops : 3.523e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   

In [ ]:
print(SAA_list_normal)
print(majority_list_normal)
print(SAA_obj_list_normal)
print(majority_obj_list_normal)